# LING 380 - Assignment 1 (Group 13)

### Group Members & Responsibilities

| Name (Last, First) | Student ID | Section Contributed | Section Edited | Other Contributions |
|---|---|---|---|---|
| Intanon, Supamongkol | 301541005 | Data collection, cleaning | Code review | Set up GitHub repo |
| Matthew Miguel  | 301422631 | Analysis code (token count, diversity, frequency) | Final formatting | Group communication |
| Member 3  |(student number) |absent | absent | absent |

## responsibility section can be change later


# Import Libraries

In [2]:
!pip install nltk
import os
import csv
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk import FreqDist
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_eng to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_eng.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_ru.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_rus to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |  

True

## Data Collection Process

| Genre | Source | Link |
|---|---|---|
| Fiction | Project Gutenburg - Pride and Prejudice, Jane Austen | [https://www.gutenberg.org/ebooks/1342](https://www.gutenberg.org/ebooks/1342) |
| Academic Essays | Project Gutenburg - Humanistic Studies of the University of Kansas | [https://www.gutenberg.org/ebooks/51685](https://www.gutenberg.org/ebooks/51685) |
| Autobiographies | Project Gutenburg - My Life Vol 1, Richard Wagner | [https://www.gutenberg.org/ebooks/5197](https://www.imsdb.com) |

#Function to Clean Files

In [3]:
print(os.path.exists("./data/pg1342.txt"))
print(os.path.exists("./data/pg51685.txt"))
print(os.path.exists("./data/pg5197.txt"))

True
True
True


In [ ]:
def remove_gutenberg_metadata(text):
    text = re.sub(r'^.*?(\*{3} START OF THE PROJECT GUTENBERG EBOOK (PRIDE AND PREJUDICE \*{3}|HUMANISTIC STUDIES OF THE UNIVERSITY OF KANSAS, VOL. 1 \*{3}|MY LIFE — VOLUME 1\*{3}))', r'\1', text, flags=re.DOTALL)

    return text
book1 = "./data/pg1342.txt"
book2 = "./data/pg51685.txt"
book3 = "./data/pg5197.txt"

def clean(book1, book2, book3):
    with open(book1, "r", encoding="utf-8") as f1:
        variable_book1 = f1.read()

    with open(book2, "r", encoding="utf-8") as f2:
        variable_book2 = f2.read()

      with open(book3, "r", encoding="utf-8") as f3:
        variable_book3 = f3.read()


    variable_book = variable_book1 + variable_book2 + variable_book3

    cleaned_text = remove_gutenberg_metadata(variable_book)
    return cleaned_text

cleaned_text = clean(book1, book2, book3)

print(cleaned_text)


#Function to Read Files into directory then  call the Clean function

In [ ]:
def read_and_clean_files(directory):
    files = [f for f in os.listdir(directory) if os.path.isfile(os.path.join(directory, f))]
    files.sort()
    combined_cleaned_text = ""
    for i in range(0, len(files), 3):
        if i + 1 < len(files):
            book1 = os.path.join(directory, files[i])
            book2 = os.path.join(directory, files[i + 1])
            book3 = os.path.join(directory, files[i + 2])


            cleaned_text = clean(book1, book2, book3)
            combined_cleaned_text += cleaned_text + "\n"

    return combined_cleaned_text

directory_path = './data'


cleaned_combined_text = read_and_clean_files(directory_path)


print(cleaned_combined_text[:1000000])

# Function to Write into a New Cleaned Text File

In [ ]:
cleaned_combined_text = read_and_clean_files(directory_path)

def write_cleaned_text_to_txt(cleaned_text, output_file):
    with open(output_file, mode='w', encoding='utf-8',) as file:
        file.write(cleaned_text)

output_txt_file = './data/cleaned_text.txt'

write_cleaned_text_to_txt(cleaned_combined_text, output_txt_file)

print(f"Cleaned text has been written to {output_txt_file}")

## Data Processing Summary

- Raw files were manually collected and placed in `raw_data/`.
- Files were combined into single files per genre in `combined_data/`.
- Files were cleaned (removing HTML, headers, extra spacing) and saved to `data/`.
- These cleaned files are ready for analysis.
